In [89]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import read_csv
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten ,MaxPooling1D
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import predict
import optuna

In [90]:
dataset=pd.read_csv('SolarPrediction_aligned_Sun.csv',engine='python',nrows=576*15)
dataset=dataset.drop("Data",axis=1)
dataset=dataset.drop("Time",axis=1)
# dataset=dataset.drop("TimeSunRise",axis=1)
# dataset=dataset.drop("TimeSunSet",axis=1)
target=dataset["Radiation"]
dataset.head(5)


,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,8,2.58,51,30.43,103,77.27,11.25,22020,67080
1,310,2.83,51,30.43,103,153.44,9.00,22020,67080
2,1206,2.16,51,30.43,103,142.04,7.87,22020,67080
3,1505,2.21,51,30.43,103,144.12,18.00,22020,67080
4,1809,2.25,51,30.43,103,67.42,11.25,22020,67080


In [91]:
dataset=dataset.values

dataset=dataset.astype('float32')

scalar1=MinMaxScaler(feature_range=(0,1))
scalar2=MinMaxScaler(feature_range=(0,1))
scalar_dim=dataset[:,1]
dataset=scalar1.fit_transform(dataset)

scalar_dim=scalar_dim.reshape(len(dataset),1)
print(scalar_dim)
scalar_dim=scalar2.fit_transform(scalar_dim)
print(scalar_dim)


[[  2.58]
 [  2.83]
 [  2.16]
 ...
 [344.9 ]
 [365.66]
 [355.57]]
[[0.00089369]
 [0.00104993]
 [0.00063121]
 ...
 [0.21482898]
 [0.2278031 ]
 [0.22149728]]


In [92]:
train_size=int(len(dataset)*0.67)
test_size=len(dataset)- train_size
train,test=dataset[0:train_size,:],dataset[train_size:len(dataset),:]

print(train.shape)
print(train)
print(test.shape)
#print(test)

(5788, 9)
[[5.8056499e-05 8.9368853e-04 3.3333325e-01 ... 5.5555558e-01
  0.0000000e+00 1.0000000e+00]
 [3.5646691e-03 1.0499278e-03 3.3333325e-01 ... 4.4444445e-01
  0.0000000e+00 1.0000000e+00]
 [1.3968393e-02 6.3120667e-04 3.3333325e-01 ... 3.8864198e-01
  0.0000000e+00 1.0000000e+00]
 ...
 [1.7786188e-01 4.3747015e-05 2.6666665e-01 ... 2.2222222e-01
  7.1428680e-01 2.9999924e-01]
 [1.8134527e-01 5.6246121e-05 2.6666665e-01 ... 2.7753085e-01
  7.1428680e-01 2.9999924e-01]
 [1.8480544e-01 6.2495703e-05 2.6666665e-01 ... 2.2222222e-01
  7.1428680e-01 2.9999924e-01]]
(2852, 9)


In [93]:
def create_dataset(dataset,look_back):
    dataX,dataY=[],[]
    for i in range(len(dataset)-look_back-1):
        #a=dataset第0行i~i+look_back個
        a=dataset[i:(i+look_back),0:9]
        dataX.append(a)
        dataY.append(dataset[i+look_back,1])
    return np.array(dataX),np.array(dataY)

    

In [94]:
#根據前面幾個資料來看下一步 彙整成look_back行的資料
#X用以輸入 Y用用以predict
look_back=5
trainX,trainY=create_dataset(train,look_back)
testX,testY=create_dataset(test,look_back)
print(testX.shape)
#print(testX)
print(testY.shape)
print(trainX.shape)
print(trainY.shape)


(2846, 5, 9)
(2846,)
(5782, 5, 9)
(5782,)


In [95]:
def objective(trial):
    model = Sequential()

    # 定義 Conv1D 層數和參數
    model.add(Conv1D(
        filters=trial.suggest_int("filters", 16, 128),
        kernel_size=3,
        activation="relu",
        input_shape=(look_back, 9)
    ))

    # 添加 MaxPooling1D 層
    model.add(MaxPooling1D(pool_size=2))

    # Flatten 層
    model.add(Flatten())

    # 全連接層
    model.add(Dense(
        units=trial.suggest_int("units", 32, 256),
        activation="relu"
    ))

    # 輸出層
    model.add(Dense(units=1, activation="linear"))

    model.compile(optimizer="adam", loss="mean_squared_error")

    # 將資料轉換成適合模型的形狀
    # X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    # X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # 訓練模型
    model.fit(trainX, trainY, epochs=10, batch_size=trial.suggest_int("batch_size", 8, 32), verbose=2)

    # 進行預測
    y_pred = model.predict(testX)

    # 計算均方誤差
    mse = mean_squared_error(testY, y_pred)

    return mse

In [96]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

# 取得最佳超參數
best_params = study.best_params
print("Best Hyperparameters:", best_params)

[I 2024-01-29 19:24:44,957] A new study created in memory with name: no-name-f6cb480d-65ca-4a75-af83-f24d44eaa2a4


Epoch 1/10
223/223 - 1s - loss: 0.0067 - 1s/epoch - 5ms/step
Epoch 2/10
223/223 - 0s - loss: 0.0051 - 399ms/epoch - 2ms/step
Epoch 3/10
223/223 - 0s - loss: 0.0051 - 417ms/epoch - 2ms/step
Epoch 4/10
223/223 - 0s - loss: 0.0048 - 410ms/epoch - 2ms/step
Epoch 5/10
223/223 - 0s - loss: 0.0046 - 393ms/epoch - 2ms/step
Epoch 6/10
223/223 - 0s - loss: 0.0045 - 404ms/epoch - 2ms/step
Epoch 7/10
223/223 - 0s - loss: 0.0045 - 371ms/epoch - 2ms/step
Epoch 8/10
223/223 - 0s - loss: 0.0044 - 362ms/epoch - 2ms/step
Epoch 9/10
223/223 - 0s - loss: 0.0043 - 368ms/epoch - 2ms/step
Epoch 10/10
223/223 - 0s - loss: 0.0043 - 360ms/epoch - 2ms/step
89/89 [==============================] - 0s 2ms/step


[I 2024-01-29 19:24:49,940] Trial 0 finished with value: 0.004451456945389509 and parameters: {'filters': 83, 'units': 134, 'batch_size': 26}. Best is trial 0 with value: 0.004451456945389509.


Epoch 1/10
482/482 - 1s - loss: 0.0090 - 1s/epoch - 3ms/step
Epoch 2/10
482/482 - 1s - loss: 0.0052 - 770ms/epoch - 2ms/step
Epoch 3/10
482/482 - 1s - loss: 0.0049 - 742ms/epoch - 2ms/step
Epoch 4/10
482/482 - 1s - loss: 0.0049 - 806ms/epoch - 2ms/step
Epoch 5/10
482/482 - 1s - loss: 0.0047 - 821ms/epoch - 2ms/step
Epoch 6/10
482/482 - 1s - loss: 0.0045 - 792ms/epoch - 2ms/step
Epoch 7/10
482/482 - 1s - loss: 0.0045 - 818ms/epoch - 2ms/step
Epoch 8/10
482/482 - 1s - loss: 0.0045 - 789ms/epoch - 2ms/step
Epoch 9/10
482/482 - 1s - loss: 0.0045 - 802ms/epoch - 2ms/step
Epoch 10/10
482/482 - 1s - loss: 0.0044 - 731ms/epoch - 2ms/step
89/89 [==============================] - 0s 1ms/step


[I 2024-01-29 19:24:58,827] Trial 1 finished with value: 0.004767263773828745 and parameters: {'filters': 23, 'units': 56, 'batch_size': 12}. Best is trial 0 with value: 0.004451456945389509.


...

Epoch 1/10
386/386 - 1s - loss: 0.0071 - 1s/epoch - 3ms/step
Epoch 2/10
386/386 - 1s - loss: 0.0050 - 613ms/epoch - 2ms/step
Epoch 3/10
386/386 - 1s - loss: 0.0048 - 619ms/epoch - 2ms/step
Epoch 4/10
386/386 - 1s - loss: 0.0048 - 617ms/epoch - 2ms/step
Epoch 5/10
386/386 - 1s - loss: 0.0047 - 614ms/epoch - 2ms/step
Epoch 6/10
386/386 - 1s - loss: 0.0046 - 656ms/epoch - 2ms/step
Epoch 7/10
386/386 - 1s - loss: 0.0044 - 648ms/epoch - 2ms/step
Epoch 8/10
386/386 - 1s - loss: 0.0045 - 686ms/epoch - 2ms/step
Epoch 9/10
386/386 - 1s - loss: 0.0044 - 675ms/epoch - 2ms/step
Epoch 10/10
386/386 - 1s - loss: 0.0044 - 693ms/epoch - 2ms/step
89/89 [==============================] - 0s 2ms/step


[I 2024-01-29 19:25:59,560] Trial 8 finished with value: 0.00659228814765811 and parameters: {'filters': 102, 'units': 44, 'batch_size': 15}. Best is trial 5 with value: 0.004135522060096264.


Epoch 1/10
276/276 - 1s - loss: 0.0067 - 1s/epoch - 4ms/step
Epoch 2/10
276/276 - 0s - loss: 0.0052 - 494ms/epoch - 2ms/step
Epoch 3/10
276/276 - 0s - loss: 0.0046 - 493ms/epoch - 2ms/step
Epoch 4/10
276/276 - 0s - loss: 0.0048 - 463ms/epoch - 2ms/step
Epoch 5/10
276/276 - 0s - loss: 0.0046 - 461ms/epoch - 2ms/step
Epoch 6/10
276/276 - 0s - loss: 0.0046 - 467ms/epoch - 2ms/step
Epoch 7/10
276/276 - 0s - loss: 0.0046 - 459ms/epoch - 2ms/step
Epoch 8/10
276/276 - 0s - loss: 0.0043 - 462ms/epoch - 2ms/step
Epoch 9/10
276/276 - 0s - loss: 0.0043 - 459ms/epoch - 2ms/step
Epoch 10/10
276/276 - 0s - loss: 0.0042 - 463ms/epoch - 2ms/step
89/89 [==============================] - 0s 2ms/step


[I 2024-01-29 19:26:05,397] Trial 9 finished with value: 0.003980303183197975 and parameters: {'filters': 111, 'units': 148, 'batch_size': 21}. Best is trial 9 with value: 0.003980303183197975.


Best Hyperparameters: {'filters': 111, 'units': 148, 'batch_size': 21}
